In [6]:
import vertexai
from vertexai.preview import reasoning_engines


PROJECT_ID = "gde-access"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://alarcon_agent_bucket"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [8]:
agent_engine = vertexai.agent_engines.get('projects/gde-access/locations/us-central1/reasoningEngines/8979746648418680832')
agent_engine.delete(force=True)

Delete Agent Engine backing LRO: projects/744349094145/locations/us-central1/operations/4420117025188216832
Agent Engine deleted. Resource name: projects/gde-access/locations/us-central1/reasoningEngines/8979746648418680832


In [4]:
from vertexai import agent_engines

agent = agent_engines.get("8979746648418680832")

/home/alarcon7a/anaconda3/envs/env_3_10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(
/home/alarcon7a/anaconda3/envs/env_3_10/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
agent.operation_schemas()

In [ ]:
import os
import json
import requests
from fastapi import FastAPI, Request
from google import auth as google_auth
from google.auth.transport import requests as google_requests

# ========= ENV REQUERIDAS =========
#  - D360_API_KEY: tu API key de 360dialog (D360-API-KEY)
#  - GCP_PROJECT:    ID de tu proyecto GCP
#  - GCP_LOCATION:   region (p.ej. "us-central1")
#  - REASONING_ENGINE_ID: ID del Reasoning Engine (solo el id numérico o nombre)
#  - WHATSAPP_VERIFY_TOKEN: token que tú inventas para verificación del webhook (opcional pero recomendado)
#
# Autenticación GCP (elige una):
#  A) ADC local: ejecuta `gcloud auth application-default login`
#  B) SERVICE ACCOUNT: exporta GOOGLE_APPLICATION_CREDENTIALS=/ruta/sa.json

D360_API_KEY = 'khSBsCpyk1Ex4xDO6bMvwAnSAK'
GCP_PROJECT = "gde-access"
GCP_LOCATION = "us-central1"
REASONING_ENGINE_ID = 'projects/744349094145/locations/us-central1/reasoningEngines/2151726663371587584'
WHATSAPP_VERIFY_TOKEN = os.getenv("WHATSAPP_VERIFY_TOKEN", "cambia_esto")

assert D360_API_KEY, "Falta D360_API_KEY"
assert GCP_PROJECT, "Falta GCP_PROJECT"
assert REASONING_ENGINE_ID, "Falta REASONING_ENGINE_ID"


In [ ]:
from typing import Any, Dict, Generator, Optional, Union


In [ ]:
def gcp_access_token() -> str:
    """Obtiene access_token con Application Default Credentials (ADC)."""
    credentials, _ = google_auth.default()
    req = google_requests.Request()
    credentials.refresh(req)
    return credentials.token

def agent_query(user_text: str) -> str:
    """Llama al Agent Engine de Vertex AI (modo síncrono :query)."""
    url = (f"https://{GCP_LOCATION}-aiplatform.googleapis.com/v1/{REASONING_ENGINE_ID}:query")
    payload = {
        "class_method": "stream_query",
        "input": {"message": user_text,
                  "user_id": "u_456",}
    }
    headers = {
        "Authorization": f"Bearer {gcp_access_token()}",
        "Content-Type": "application/json; charset=utf-8"
    }
    r = requests.post(url, headers=headers, data=json.dumps(payload), timeout=60)
    r.raise_for_status()
    data = r.json()
    return data.get("output") or str(data)

In [ ]:
def agent_stream_query(
    message: Union[str, Dict[str, Any]],
    user_id: str,
    session_id: Optional[str] = None,
    **kwargs: Dict[str, Any],
) -> Generator[Dict[str, Any], None, None]:
    """
    Llama al endpoint :streamQuery (SSE) del Agent Engine y va rindiendo eventos.
    Respeta el esquema:
      - required: message, user_id
      - optional: session_id
      - **kwargs: se envían en el payload para el runner
    Yields: dict con el contenido de cada "data:" del SSE (parseado como JSON).
    """
    assert GCP_PROJECT and REASONING_ENGINE_ID, "Falta configurar GCP_PROJECT/REASONING_ENGINE_ID"

    url = (f"https://{GCP_LOCATION}-aiplatform.googleapis.com/v1/projects/{GCP_PROJECT}"
           f"/locations/{GCP_LOCATION}/reasoningEngines/{REASONING_ENGINE_ID}:streamQuery")

    payload: Dict[str, Any] = {
        "message": message,
        "user_id": user_id,
    }
    if session_id is not None:
        payload["session_id"] = session_id
    if kwargs:
        payload.update(kwargs)

    headers = {
        "Authorization": f"Bearer {gcp_access_token()}",
        "Content-Type": "application/json; charset=utf-8",
        "Accept": "text/event-stream",
    }

    with requests.post(url, headers=headers, data=json.dumps(payload), stream=True, timeout=300) as r:
        r.raise_for_status()
        for raw in r.iter_lines(decode_unicode=True):
            if not raw:
                continue
            # Formato SSE típico: "data: {...json...}"
            if raw.startswith("data:"):
                try:
                    evt = json.loads(raw[len("data:"):].strip())
                except Exception:
                    # Algunos proveedores mandan "data: [DONE]" u otros marcadores
                    if raw.strip().endswith("[DONE]"):
                        break
                    continue
                yield evt

def collect_stream_to_text(
    message: Union[str, Dict[str, Any]],
    user_id: str,
    session_id: Optional[str] = None,
    **kwargs: Dict[str, Any],
) -> str:
    """
    Consume agent_stream_query y devuelve un solo string.
    Intenta extraer delta/choices/output según lo que emita el runner.
    """
    parts: list[str] = []
    for evt in agent_stream_query(message=message, user_id=user_id, session_id=session_id, **kwargs):
        # Heurísticas comunes de eventos:
        # - evt.get("delta")           -> fragmento parcial
        # - evt.get("output")          -> bloque final o parcial
        # - evt.get("text") / "token"  -> algunos runners emiten así
        for key in ("delta", "output", "text", "token"):
            val = evt.get(key)
            if isinstance(val, str):
                parts.append(val)
    return "".join(parts).strip()

In [ ]:
gcp_access_token()

In [ ]:
from google import auth as google_auth
from google.auth.transport import requests as google_requests
import requests
import json

def get_identity_token():
    credentials, _ = google_auth.default()
    auth_request = google_requests.Request()
    credentials.refresh(auth_request)
    return credentials.token

# Hacer request con streaming
response = requests.post(
    f"https://us-central1-aiplatform.googleapis.com/v1/projects/744349094145/locations/us-central1/reasoningEngines/2151726663371587584:streamQuery",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {get_identity_token()}",
    },
    data=json.dumps({
        "class_method": "stream_query",
        "input": {
            "message": "Cual es la hora en newyork?",
            "user_id": "user_1234",  # Requerido
        }
    }),
    stream=True
)

# Procesar respuestas en streaming
for line in response.iter_lines():
    if line:
        print(line.decode('utf-8'))

In [ ]:
def agent_query_stream(user_text: str, user_id: str, session_id: str = None) -> str:
    """Llama al Agent Engine de Vertex AI (modo streaming: stream_query)."""
    url = f"https://us-central1-aiplatform.googleapis.com/v1/projects/744349094145/locations/us-central1/reasoningEngines/2151726663371587584:streamQuery"
    payload = {
        "class_method": "stream_query",
        "input": {
            "message": user_text,
            "user_id": user_id,
            "session_id": session_id  # Opcional, se crea automáticamente si es None
        }
    }
    headers = {
        "Authorization": f"Bearer {get_identity_token()}",
        "Content-Type": "application/json"
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(payload), stream=True, timeout=60)
    response.raise_for_status()
    
    # Recopilar todas las respuestas del stream
    full_response = ""
    final_output = None
    
    # Procesar cada línea del stream
    for line in response.iter_lines():
        if line:
            try:
                line_str = line.decode('utf-8').strip()
                if line_str:  # Si la línea no está vacía
                    data = json.loads(line_str)
                    
                    # Buscar respuesta final en content.parts
                    if 'content' in data and 'parts' in data['content']:
                        for part in data['content']['parts']:
                            # Si es una respuesta de texto (respuesta final)
                            if 'text' in part and data['content'].get('role') == 'model':
                                final_response = part['text']
                            
            except (json.JSONDecodeError, UnicodeDecodeError, KeyError):
                continue
    
    return final_response or "No se recibió respuesta válida"

In [ ]:
agent_query_stream("whats the weather in new york", user_id="u_456")